In [4]:
import torch
import numpy as np
from Networks.mesh2mesh import *
from Networks.rgbd2mesh import RGBD2Mesh
import trimesh
from Networks.obj2pointcloud import *
from Networks.pointclouddataset import *
from chamferdist import ChamferDistance
from Networks.foldingnet import FoldingNetDec
from Networks import train_mesh2mesh

In [2]:
####1 get data for stage one autoencoder training

train_dataset = pcd_stage1(split='train')
train_dataloader = torch.utils.data.DataLoader(
        train_dataset,   # Datasets return data one sample at a time; Dataloaders use them and aggregate samples into batches
        batch_size=1,   # The size of batches is defined here
        shuffle=True,    # Shuffling the order of samples is useful during training to prevent that the network learns to depend on the order of the input data
        num_workers=4,   # Data is usually loaded in parallel by num_workers
        pin_memory=True  # This is an implementation detail to speed up data uploading to the GPU
    )
val_dataset = pcd_stage1(split='val')
val_dataloader = torch.utils.data.DataLoader(
        val_dataset,     # Datasets return data one sample at a time; Dataloaders use them and aggregate samples into batches
        batch_size=1,   # The size of batches is defined here
        shuffle=False,   # During validation, shuffling is not necessary anymore
        num_workers=4,   # Data is usually loaded in parallel by num_workers
        pin_memory=True  # This is an implementation detail to speed up data uploading to the GPU
    )    

In [3]:
##Be carefull removing the model



config = {
    'experiment_name': 'pcd_stage1_overfit',
    'device': 'cuda:0',                   # change this to cpu if you do not have a GPU
    'is_overfit': True,                   # True since we're doing overfitting
    'batch_size': 1,
    'resume_ckpt_en': None,
    'resume_ckpt_de': None,
    'learning_rate': 0.0001,
    'max_epochs': 300,
    'print_every_n': 100,
    'validate_every_n': 100,
}

In [4]:



train_mesh2mesh.main(config)


Using device: cuda:0
[001/00019] train_loss: 0.0189191915
[001/00019] val_loss: 0.0065100389
[002/00039] train_loss: 0.0066918780
[002/00039] val_loss: 0.0057656527
[003/00059] train_loss: 0.0055907195
[003/00059] val_loss: 0.0055175549
[004/00079] train_loss: 0.0065607616
[004/00079] val_loss: 0.0057122614
[006/00019] train_loss: 0.0062096286
[006/00019] val_loss: 0.0061137844
[007/00039] train_loss: 0.0052823322
[007/00039] val_loss: 0.0050643082
[008/00059] train_loss: 0.0059039012
[008/00059] val_loss: 0.0051013329
[009/00079] train_loss: 0.0047843145
[009/00079] val_loss: 0.0049696155
[011/00019] train_loss: 0.0048867470
[011/00019] val_loss: 0.0057279938
[012/00039] train_loss: 0.0047318251
[012/00039] val_loss: 0.0037244379
[013/00059] train_loss: 0.0034602307
[013/00059] val_loss: 0.0027869330
[014/00079] train_loss: 0.0027653593
[014/00079] val_loss: 0.0023355766
[016/00019] train_loss: 0.0024237112
[016/00019] val_loss: 0.0023658171
[017/00039] train_loss: 0.0025202188
[017/0

KeyboardInterrupt: 

In [5]:
torch.cuda.empty_cache()

In [2]:
config = {
    'experiment_name': 'pcd_stage1',
    'device': 'cuda:0',                   # change this to cpu if you do not have a GPU
    'is_overfit': False,                   # True since we're doing overfitting
    'batch_size': 1,
    'resume_ckpt_en': '/home/wuha/Hanfeng/TUM/ML3D/ML3Dfinal_2021WS/models/runs/pcd_stage1/encoder_best.pth',
    'resume_ckpt_de': '/home/wuha/Hanfeng/TUM/ML3D/ML3Dfinal_2021WS/models/runs/pcd_stage1/decoder_best.pth',
    #'resume_ckpt_en': None,
    #'resume_ckpt_de': None,
    'learning_rate': 0.0001,
    'max_epochs': 1000,
    'print_every_n': 100,
    'validate_every_n': 100,
}
train_mesh2mesh.main(config)

Using device: cuda:0
[000/00099] train_loss: 0.0007168060
[000/00099] val_loss: 0.0006033471
[000/00199] train_loss: 0.0007950362
[000/00199] val_loss: 0.0007556513
[000/00299] train_loss: 0.0007420536
[000/00299] val_loss: 0.0006384985
[000/00399] train_loss: 0.0008447172
[000/00399] val_loss: 0.0006559506
[000/00499] train_loss: 0.0009390969
[000/00499] val_loss: 0.0006362721
[000/00599] train_loss: 0.0006838169
[000/00599] val_loss: 0.0006181712
[000/00699] train_loss: 0.0007863503
[000/00699] val_loss: 0.0006372426
[000/00799] train_loss: 0.0007451214
[000/00799] val_loss: 0.0006312531
[000/00899] train_loss: 0.0007174762
[000/00899] val_loss: 0.0005934975
[000/00999] train_loss: 0.0007488182
[000/00999] val_loss: 0.0006222363
[000/01099] train_loss: 0.0008030093
[000/01099] val_loss: 0.0006594332
[000/01199] train_loss: 0.0007632369
[000/01199] val_loss: 0.0006239315
[000/01299] train_loss: 0.0007546756
[000/01299] val_loss: 0.0006130763
[000/01399] train_loss: 0.0007612381
[000/0

KeyboardInterrupt: 

In [4]:
encoder = PointNetEncoder(numpoints=40000)
encoder.load_state_dict(torch.load(f'./models/runs/{config["experiment_name"]}/encoder_best.pth'))
encoder.eval()
decoder = FoldingNetDec(200,200)
decoder.load_state_dict(torch.load(f'./models/runs/{config["experiment_name"]}/decoder_best.pth'))
decoder.eval()
val_dataset = pcd_stage1(split='val')
val_dataloader = torch.utils.data.DataLoader(
        val_dataset,     # Datasets return data one sample at a time; Dataloaders use them and aggregate samples into batches
        batch_size=1,   # The size of batches is defined here
        shuffle=False,   # During validation, shuffling is not necessary anymore
        num_workers=4,   # Data is usually loaded in parallel by num_workers
        pin_memory=True  # This is an implementation detail to speed up data uploading to the GPU
    )
x = 1
for batch in val_dataloader:
    input = batch.numpy()
    batch = batch.permute(0,2,1)
    recon = decoder(encoder(batch))
    out = recon.permute(0,2,1).detach().numpy()

    print(input.shape)
    print(out.shape)

    
    export_pointcloud_to_obj(f'./input_{x}.obj', input[0])
    export_pointcloud_to_obj(f'./output_{x}.obj', out[0])


    x +=1
    if x==10: break

    



RuntimeError: Error(s) in loading state_dict for FoldingNetDec:
	Missing key(s) in state_dict: "conv.weight", "conv.bias", "fold1.conv1.weight", "fold1.conv1.bias", "fold1.conv2.weight", "fold1.conv2.bias", "fold1.conv3.weight", "fold1.conv3.bias", "fold2.conv1.weight", "fold2.conv1.bias", "fold2.conv2.weight", "fold2.conv2.bias", "fold2.conv3.weight", "fold2.conv3.bias". 
	Unexpected key(s) in state_dict: "fc.0.weight", "fc.0.bias", "fc.2.weight", "fc.2.bias", "fc.4.weight", "fc.4.bias". 

In [4]:
loss_criterion = ChamferDistance()

encoder = PointNetEncoder(numpoints=4000)
encoder.load_state_dict(torch.load(f'./models/runs/{config["experiment_name"]}/encoder_best.pth'))
encoder.eval()
decoder = PointNetDecoder(numpoints=4000)
decoder.load_state_dict(torch.load(f'./models/runs/{config["experiment_name"]}/decoder_best.pth'))
decoder.eval()
val_dataset = pcd_stage1(split='val')
val_dataloader = torch.utils.data.DataLoader(
        val_dataset,     # Datasets return data one sample at a time; Dataloaders use them and aggregate samples into batches
        batch_size=1,   # The size of batches is defined here
        shuffle=False,   # During validation, shuffling is not necessary anymore
        num_workers=4,   # Data is usually loaded in parallel by num_workers
        pin_memory=True  # This is an implementation detail to speed up data uploading to the GPU
    )  
x=1     
for batch in val_dataloader:
    input = batch.numpy()
    batch = batch.permute(0,2,1)
    recon = decoder(encoder(batch))
    out = recon.permute(0,2,1).detach().numpy()

    print(input.shape)
    print(out.shape)
    print(loss_criterion(recon.permute(0,2,1), batch.permute(0,2,1)))
    print(recon.shape)
    print(batch.permute(0,2,1).shape)

    
    export_pointcloud_to_obj(f'./input_{x}.obj', input[0])
    export_pointcloud_to_obj(f'./output_{x}.obj', out[0])


    x +=1
    if x==10: break    

(1, 4000, 3)
(1, 4000, 3)
tensor(0.8896, grad_fn=<MeanBackward0>)
torch.Size([1, 3, 4000])
torch.Size([1, 4000, 3])
(1, 4000, 3)
(1, 4000, 3)
tensor(0.8429, grad_fn=<MeanBackward0>)
torch.Size([1, 3, 4000])
torch.Size([1, 4000, 3])
(1, 4000, 3)
(1, 4000, 3)
tensor(0.6607, grad_fn=<MeanBackward0>)
torch.Size([1, 3, 4000])
torch.Size([1, 4000, 3])
(1, 4000, 3)
(1, 4000, 3)
tensor(1.4037, grad_fn=<MeanBackward0>)
torch.Size([1, 3, 4000])
torch.Size([1, 4000, 3])
(1, 4000, 3)
(1, 4000, 3)
tensor(1.7110, grad_fn=<MeanBackward0>)
torch.Size([1, 3, 4000])
torch.Size([1, 4000, 3])
(1, 4000, 3)
(1, 4000, 3)
tensor(1.4772, grad_fn=<MeanBackward0>)
torch.Size([1, 3, 4000])
torch.Size([1, 4000, 3])
(1, 4000, 3)
(1, 4000, 3)
tensor(0.5348, grad_fn=<MeanBackward0>)
torch.Size([1, 3, 4000])
torch.Size([1, 4000, 3])
(1, 4000, 3)
(1, 4000, 3)
tensor(0.9369, grad_fn=<MeanBackward0>)
torch.Size([1, 3, 4000])
torch.Size([1, 4000, 3])
(1, 4000, 3)
(1, 4000, 3)
tensor(0.6032, grad_fn=<MeanBackward0>)
torch.

In [5]:
loss_criterion = torch.nn.MSELoss()

encoder = PointNetEncoder(return_point_features=True, numpoints=40000)
encoder.load_state_dict(torch.load(f'./models/runs/{config["experiment_name"]}/encoder_best.pth'))
encoder.eval()
decoder = PointNetDecoder()
decoder.load_state_dict(torch.load(f'./models/runs/{config["experiment_name"]}/decoder_best.pth'))
decoder.eval()
val_dataset = pcd_stage1(split='overfit')
val_dataloader = torch.utils.data.DataLoader(
        val_dataset,     # Datasets return data one sample at a time; Dataloaders use them and aggregate samples into batches
        batch_size=1,   # The size of batches is defined here
        shuffle=False,   # During validation, shuffling is not necessary anymore
        num_workers=4,   # Data is usually loaded in parallel by num_workers
        pin_memory=True  # This is an implementation detail to speed up data uploading to the GPU
    )  
x=1     
for batch in val_dataloader:
    input = batch.numpy()
    batch = batch.permute(0,2,1)
    recon = decoder(encoder(batch))
    out = recon.detach().numpy()

    print(input.shape)
    print(out.shape)
    print(loss_criterion(recon, batch.permute(0,2,1)))
    print(recon.shape)
    print(batch.permute(0,2,1).shape)

    
    export_pointcloud_to_obj(f'./input_{x}.obj', input[0])
    export_pointcloud_to_obj(f'./output_{x}.obj', out[0])


    x +=1
    if x==10: break    

RuntimeError: Error(s) in loading state_dict for PointNetDecoder:
	size mismatch for fc.4.weight: copying a param with shape torch.Size([120000, 1024]) from checkpoint, the shape in current model is torch.Size([3072, 1024]).
	size mismatch for fc.4.bias: copying a param with shape torch.Size([120000]) from checkpoint, the shape in current model is torch.Size([3072]).

In [1]:
### stage 2 training

config = {
    'experiment_name': 'pcd_stage2',
    'device': 'cuda:0',                   # change this to cpu if you do not have a GPU
    'is_overfit': False,                   # True since we're doing overfitting
    'batch_size': 1,
    'resume_ckpt_en': '/home/wuha/Hanfeng/TUM/ML3D/ML3Dfinal_2021WS/models/runs/pcd_stage2/encoder_best.pth',
    #'resume_ckpt_en': None,
    'encoder_pretrained': '/home/wuha/Hanfeng/TUM/ML3D/ML3Dfinal_2021WS/models/runs/pcd_stage1/encoder_best.pth', 
    'learning_rate': 0.0001,
    'max_epochs': 300,
    'print_every_n': 100,
    'validate_every_n': 1000,
}


In [2]:
from Networks import train_mesh2mesh_stage2


train_mesh2mesh_stage2.main(config)

Using device: cuda:0
[000/00099] train_loss: 0.0013866251
[000/00199] train_loss: 0.0005798664
[000/00299] train_loss: 0.0005497261
[000/00399] train_loss: 0.0005865927
[000/00499] train_loss: 0.0006226039
[000/00599] train_loss: 0.0005177707
[000/00699] train_loss: 0.0005058221
[000/00799] train_loss: 0.0005430110
[000/00899] train_loss: 0.0004930555
[000/00999] train_loss: 0.0004593548
[000/00999] val_loss: 0.0004580859
[000/01099] train_loss: 0.0004883460
[000/01199] train_loss: 0.0004735059
[000/01299] train_loss: 0.0004305126
[000/01399] train_loss: 0.0004026025
[000/01499] train_loss: 0.0004288288
[000/01599] train_loss: 0.0004276766
[000/01699] train_loss: 0.0003645228
[000/01799] train_loss: 0.0003398186
[000/01899] train_loss: 0.0003673212
[000/01999] train_loss: 0.0003674010
[000/01999] val_loss: 0.0003338145
[000/02099] train_loss: 0.0003216727
[000/02199] train_loss: 0.0003686877
[000/02299] train_loss: 0.0003300356
[000/02399] train_loss: 0.0003108314
[000/02499] train_los

KeyboardInterrupt: 

In [6]:
encoder = PointNetEncoder(numpoints=1000)
encoder.load_state_dict(torch.load(f'./models/runs/pcd_stage2/encoder_stage2_best.pth'))
encoder.eval()
decoder = PointNetDecoder(numpoints=4000)
decoder.load_state_dict(torch.load(f'./models/runs/pcd_stage1/decoder_best.pth'))
decoder.eval()
val_dataset = pcd_stage2(split='val')
val_dataloader = torch.utils.data.DataLoader(
        val_dataset,     # Datasets return data one sample at a time; Dataloaders use them and aggregate samples into batches
        batch_size=1,   # The size of batches is defined here
        shuffle=False,   # During validation, shuffling is not necessary anymore
        num_workers=4,   # Data is usually loaded in parallel by num_workers
        pin_memory=True  # This is an implementation detail to speed up data uploading to the GPU
    )  
x=1     
for batch in val_dataloader:
    gt = batch['full'].numpy()
    input = batch['partial'].permute(0,2,1)
    print(gt.shape)
    print(input.shape)
    recon = decoder(encoder(input))
    out = recon.detach().numpy()




    print(out.shape)

    
    export_pointcloud_to_obj(f'./full_{x}.obj', gt[0])
    export_pointcloud_to_obj(f'./partial_{x}.obj', out[0])


    x +=1
    if x==10: break 

AttributeError: 'dict' object has no attribute 'numpy'